# Module 2 - Problem 1: Balancing a Chemical Reaction

This problem consists in look for the coefficients to balance the chemical reaction with the solution a set of lineal equation

                        X = A⁻¹b
Where:
        A is a square matrix with the amount of the atoms in each molecule
        b is a vector
        X is a vector with the coefficients to balance the chemical reaction

In [142]:
from sys import path

path.append("../../src/m1")

import numpy as np
from atomic_mass_percentage import split_symbols_amount, molecular_mass


In [40]:
def input_substance(num_react: int, num_prod: int) -> list[str]:

    list_react_prod: list[str] = []
    for r in range(num_react):
        list_react_prod.append(input(f"what is the {r+1} reactive?"))

    for p in range(num_prod):
        list_react_prod.append(input(f"what is the {p+1} product?"))

    return list_react_prod

In [31]:
def molecule_composition(molecule: list) -> dict[str : int]:
    for m in molecule:
        yield split_symbols_amount(m)

In [73]:
def matrix_amount_atom(atoms: set[str], comp_react_prod: list[dict[str : int]], num_r: int) -> np.array:

    matrix_amount_atoms: list[list[int]] = []
    for a in atoms:            
        temp: list[int] = []
        for count, molecule in enumerate(comp_react_prod):
            if a in molecule.keys() and count < num_r:
                temp.append(molecule[a])
            elif a in molecule.keys() and count >= num_r:
                temp.append(-molecule[a])
            else:
                temp.append(0)
        matrix_amount_atoms.append(temp)

    return np.array(matrix_amount_atoms,dtype=np.float_)

In [148]:
def heaviest(molecules: list[dict[str : int]]) -> int:

    heaviest_index: int = 0
    heaviest: float = 0.
    for count, mol in enumerate(molecules):
        mass_mol: float = molecular_mass(mol)
        if mass_mol > heaviest:
            heaviest = mass_mol
            heaviest_index = count

    return heaviest_index

In [146]:
def add_row_col(matrix: np.array, heaviest: int, nrow: int, ncol: int) -> np.array:

    if nrow > ncol:
        column_add: np.array = np.array([0 if i < (nrow - 1) else 1 for i in range(nrow)], dtype=np.float_)
        return np.column_stack((matrix, column_add))
    else:
        row_add: np.array = np.array([1 if i == heaviest else 0 for i in range(ncol)], dtype=np.float_)
        return np.r_[matrix,[row_add]]


In [95]:
def balance(mat_comp_mol: np.array, coef_result: np.array) -> np.array:

    return np.linalg.solve(mat_comp_mol, coef_result)

In [153]:
num_react: int = int(input("how many reactives are there? "))
num_prod: int = int(input("how many products are there? "))
list_react_prod = input_substance(num_react,num_prod)

In [154]:
comp_react_prod: list[dict[str : int]] = [comp for comp in molecule_composition(list_react_prod)]

In [155]:
atoms: set[str] = {atom for molecule in comp_react_prod[:num_react] for atom in molecule.keys()}
matrix_amount_atoms = matrix_amount_atom(atoms, comp_react_prod, num_react)

In [156]:
row, col = matrix_amount_atoms.shape
if row == col:
    print(f"The matrix with amount of the atoms is square:  {row} x {col}")
    m_square_amount_atoms = matrix_amount_atoms
    coef_result: np.array = np.array([0 for i in range(row)], dtype=np.float_)
else:
    print(f"The matrix with amount of the atoms is not square:  {row} x {col}")
    message: str = 'col'
    n: int = row
    if col > row:
        message = 'row'
        n = col
    
    print(f"Then, is add one {message} in the last")
    heaviest_molecule = heaviest(comp_react_prod)
    
    m_square_amount_atoms = add_row_col(matrix_amount_atoms, heaviest_molecule, row, col)

    coef_result: np.array = np.array([0 if i < (n-1) else 1 for i in range(n)], dtype=np.float_)

The matrix with amount of the atoms is not square:  3 x 4
Then, is add one row in the last


In [158]:
balance_coeff = balance(m_square_amount_atoms,coef_result)
print(f"Balance coeffecients from reactives to products: {balance_coeff}")

Balance coeffecients from reactives to products: [6. 6. 1. 6.]


In [159]:
for count, molecule in enumerate(list_react_prod):
    bc: str = ''
    if balance_coeff[count] > 1:
        bc = str(balance_coeff[count])
    print(bc + " " + molecule, end=" ")
    if count < num_react - 1 or count >= num_react and count < num_react + num_prod - 1:
        print("+",end=" ")
    if count == num_react - 1:
        print("-->", end=" ")

6.0 CO2 + 6.0 H2O -->  C6H12O6 + 6.0 O2 

try:
    * C2H5OH + 3O2 --> 2CO2 + 3H2O
    * Na2CO3 + 4C + N2 --> 2NaCN + 3CO
    * CO2 + H2O  -->  C6H12O6 + O2
    * FeS2 + O2  -->  Fe2O3 + SO2